# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123128e+02     1.640410e+00
 * time: 0.08849620819091797
     1     1.068246e+01     8.759217e-01
 * time: 0.910952091217041
     2    -1.180391e+01     1.074858e+00
 * time: 1.0513520240783691
     3    -3.401406e+01     7.982792e-01
 * time: 1.21864914894104
     4    -4.770256e+01     5.481225e-01
 * time: 1.3680660724639893
     5    -5.682452e+01     1.971017e-01
 * time: 1.5134031772613525
     6    -5.967678e+01     1.395010e-01
 * time: 1.6197991371154785
     7    -6.081552e+01     5.415686e-02
 * time: 1.7319371700286865
     8    -6.127445e+01     6.801012e-02
 * time: 1.84405517578125
     9    -6.158094e+01     5.243551e-02
 * time: 1.9500641822814941
    10    -6.182802e+01     3.441750e-02
 * time: 2.063599109649658
    11    -6.201123e+01     1.995612e-02
 * time: 2.169530153274536
    12    -6.207919e+01     1.816470e-02
 * time: 2.282785177230835
    13    -6.213985e+01     1.488489e-02
 * time: 2.3964250087738037


In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557679
    AtomicNonlocal      14.8522651
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336819

    total               -62.261666458885
